# **PYSPARK INTERVIEW QUESTIONS - LS**

https://www.youtube.com/watch?v=fOCiis31Ng4&t=7s

In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *
from pyspark.sql.window import Window

### 1. While ingesting customer data from an external source, you notice duplicate entries. How would you remove duplicates and retain only the latest entry based on a timestamp column?

In [0]:
data = [("101", "2023-12-01", 100), ("101", "2023-12-02", 150), 
        ("102", "2023-12-01", 200), ("102", "2023-12-02", 250)]
columns = ["product_id", "date", "sales"]

df = spark.createDataFrame(data, columns)
df.display()

**Solution**

In [0]:
# df = df.withColumn("date", to_date(col("date"), "yyyy-MM-dd"))
# df = df.orderBy('product_id',"date", ascending=[True, False]).dropDuplicates(subset=["product_id"])
# df.display()

from pyspark.sql.window import Window

window_spec = Window.partitionBy("product_id").orderBy(col("date").desc())
df_latest = df.withColumn("row_num", row_number().over(window_spec)).filter(col("row_num") == 1).drop("row_num")
df_latest.display()

### 2. While processing data from multiple files with inconsistent schemas, you need to merge them into a single DataFrame. How would you handle this inconsistency in PySpark?

In [0]:
# df = spark.read.format('parquet').option('mergeSchema', True).load("/path/workshop/raw/bronze")


### 3. What are the key difference between spark and hadop Map reduce in terms of performance and scalability

In [0]:
# 1. Performmance in spark compare to map reduce is 100 x better because of in memory processing
# 2. Hadoop is meant to do the batch processing while Spark is efficient both Streaming and Batch processing

### 4. You are working with a real-time data pipeline, and you notice missing values in your streaming data Column - Category. How would you handle null or missing values in such a scenario?

df_stream = spark.readStream.schema("id INT, value STRING").csv("path/to/stream")

In [0]:
# df_stream = df_stream.fillna({'Category':'N/A'})

### 5. You need to calculate the total number of actions performed by users in a system. How would you calculate the top 5 most active users based on this information?

In [0]:
data = [("user1", 5), ("user2", 8), ("user3", 2), ("user4", 10), ("user2", 3)]
columns = ["user_id", "actions"]

df = spark.createDataFrame(data, columns)
df.display()

**Solution**

In [0]:
df = df.groupBy("user_id").agg(sum("actions").alias("total_actions")).orderBy("total_actions",ascending=False).limit(5)
df.display()

### 6. While processing sales transaction data, you need to identify the most recent transaction for each customer. How would you approach this task?

In [0]:
data = [("cust1", "2023-12-01", 100), ("cust2", "2023-12-02", 150),
        ("cust1", "2023-12-03", 200), ("cust2", "2023-12-04", 250)]
columns = ["customer_id", "transaction_date", "sales"]
df = spark.createDataFrame(data, columns)
df.display()

**Solution**

In [0]:
from pyspark.sql.window import Window

df = df.withColumn("transaction_date", to_date("transaction_date", "yyyy-MM-dd"))
wind = Window.partitionBy('customer_id').orderBy(col('transaction_date').desc())
df = df.withColumn("rank", dense_rank().over(wind)).filter("rank = 1")
df.display()

### 7. You need to identify customers who haven’t made any purchases in the last 30 days. How would you filter such customers?

In [0]:
data = [("cust1", "2024-12-01"), ("cust2", "2025-09-10"), ("cust3", "2025-08-20")]
columns = ["customer_id", "last_purchase_date"]

df = spark.createDataFrame(data, columns)

df.display()

**Solution**

In [0]:
df = df.withColumn("last_purchase_date", to_date("last_purchase_date", "yyyy-MM-dd"))\
  .withColumn('no_days_since_last_pd', datediff(current_date(),col("last_purchase_date")))\
  .filter(col('no_days_since_last_pd')>30 )
df.display()

### 8. While analyzing customer reviews, you need to identify the most frequently used words in the feedback. How would you implement this?

In [0]:
data = [("customer1", "logesh The product is great logesh"), ("customer2", "Great product, logesh fast delivery"), ("customer3", "Not bad, logesh could be better")]
columns = ["customer_id", "feedback"]

df = spark.createDataFrame(data, columns)

df.display()


**Solution**

In [0]:
# identify the mostly used words in feedbck column
from pyspark.sql.functions import explode, split, lower, regexp_replace, col

words_df = df.withColumn(
    "word",
    explode(
        split(
            regexp_replace(lower(col("feedback")), r"[^a-zA-Z0-9\s]", ""), " "
        )
    )
).filter(col("word") != "")

word_counts = words_df.groupBy("word").count().orderBy(col("count").desc())
display(word_counts)

### 9. You need to calculate the cumulative sum of sales over time for each product. How would you approach this?

In [0]:
data = [("product1", "2023-12-01", 100), ("product2", "2023-12-02", 200),
        ("product1", "2023-12-03", 150), ("product2", "2023-12-04", 250)]
columns = ["product_id", "date", "sales"]
df = spark.createDataFrame(data, columns)
df.display()

**Solution**

In [0]:
df = df.withColumn("date", to_date("date", "yyyy-MM-dd"))
wind = Window.partitionBy('product_id').orderBy(col('date').asc())
df.withColumn('cumulative_sum', sum('sales').over(wind)).display()

### 10. While preparing a data pipeline, you notice some duplicate rows in a dataset. How would you remove the duplicates without affecting the original order?

In [0]:
data = [("John", 25), ("Jane", 30), ("John", 25), ("Alice", 22)]
columns = ["name", "age"]
df = spark.createDataFrame(data, columns)
df.display()
# df.dropDuplicates(subset=['name', 'age']).display() - > it does not quarantee the order so use monotonically_increasing_id

**Solution**

In [0]:

from pyspark.sql.functions import monotonically_increasing_id

df = df.withColumn("row_id", monotonically_increasing_id())
df.display()
df = df.dropDuplicates(subset=['name', 'age']).orderBy("row_id").drop("row_id")
df.display()

### 11. You are working with user activity data and need to calculate the average session duration per user. How would you implement this?

In [0]:
data = [("user1", "2023-12-01", 50), ("user1", "2023-12-02", 60), 
        ("user2", "2023-12-01", 45), ("user2", "2023-12-03", 75)]
columns = ["user_id", "session_date", "duration"]
df = spark.createDataFrame(data, columns)

df.display()

**Solution**

In [0]:
df.groupBy("user_id").agg(avg("duration")).display()

### 12. While analyzing sales data, you need to find the product with the highest sales for each month. How would you accomplish this?

In [0]:
data = [("product1", "2023-12-01", 100), ("product2", "2023-12-01", 150), 
        ("product1", "2023-12-02", 200), ("product2", "2023-12-02", 250)]
columns = ["product_id", "date", "sales"]
df = spark.createDataFrame(data, columns)
df.display()

**Solution**

In [0]:
df = df.withColumn('date', to_date(col('date'),'yyyy-MM-dd'))
df.groupBy(month('date').alias('month_number'), 'product_id').agg(sum('sales').alias('total_sales'))\
    .withColumn('rank', dense_rank().over(Window.partitionBy('month_number').orderBy(col('total_sales').desc())))\
    .filter(col('rank') == 1)\
    .select('product_id', 'total_sales')\
    .display()


### 13. You are working with a large Delta table that is frequently updated by multiple users. The data is stored in partitions, and sometimes updates can cause inconsistent reads due to concurrent transactions. How would you ensure ACID compliance and avoid data corruption in PySpark?

In [0]:
# Delta Lake or Delta format take care of the concurrent transaction using 'delta_logs'
# we should take care of logic to 'upcert' data correctly


### 14. You need to process a large dataset stored in PARQUET format and ensure that all columns have the right schema (Almost). How would you do this?

In [0]:
# Either you can define the schema expicitly or infer it from the data

### 15. You are reading a CSV file and need to handle corrupt records gracefully by skipping them. How would you configure this in PySpark?

In [0]:
# df = spark.read.format("csv") \
#     .option("mode", "DROPMALFORMED") \
#     .option("header", "true") \
#     .load("/path/to/your/file.csv")
# df.display()

### 16. What is the difference between `RDDs, Dataframe & Dataset`

In [0]:
# RDD (Resilient Distributed Dataset): Low-level, immutable distributed collection of objects. Supports functional transformations (map, filter, etc.), but lacks optimizations and schema awareness.
# DataFrame: Distributed collection of data organized into named columns. Provides schema, optimized execution, and SQL-like operations. Most common in PySpark.
# Dataset: Type-safe, object-oriented distributed collection available in Scala/Java. Combines RDD benefits with DataFrame optimizations. Not available in PySpark.

### 17. What is Query Optimization?

In [0]:
# Code => logical plan -> logical plan optimization (RBO) -> physical plans -> physical plan optimization (CBO) -> execution plan -> RDD
# What is Query Optimization?
# Query optimization is the process of transforming a query into an equivalent query that is more efficient to execute. It involves analyzing the query's structure and applying various techniques to find the most efficient execution plan. The goal is to minimize the amount of data processed and the time taken to execute the query.

# There are two main types of query optimization:

# Rule-based optimization (RBO): This approach applies a set of predefined rules to the query's logical plan to find the most efficient execution plan. The rules are based on the properties of the data and the query itself. The rules are applied in a specific order, and the resulting plan is evaluated for its cost. The process continues until no further improvements can be made.

# Cost-based optimization (CBO): This approach uses statistical information about the data and the query to estimate the cost of different execution plans. The cost is then used to select the most efficient plan. The process involves estimating the cost of different operators and combining them to form a plan. The plan is then evaluated for its cost, and the process continues until no further improvements can be made.



### 18. Tell me about Spark session

In [0]:
# Tell me about Spark session

# SparkSession is the entry point for Spark functionality. It provides a unified interface for working with structured and semi-structured data. It provides methods for creating DataFrames, working with Spark SQL, and managing Spark configurations.

# SparkSession is a singleton object that is created when you create a Spark application. It is used to create DataFrames, register temporary views, and manage Spark configurations.

# SparkSession is a high-level API that provides a unified interface for working with structured and semi-structured data. It provides methods for creating DataFrames, working with Spark SQL, and managing Spark configurations.


### 19. Wide and Narrow Transformations

In [0]:
# # what is Wide and Narrow Transformations?
# wide transformation: transformation that produces more output partitions than input partitions. For example, a join operation or a groupBy operation.

# narrow transformation: transformation that produces fewer output partitions than input partitions. For example, a map operation or a filter operation.

### 20. What is COALESCE() and REPARTITION()?

In [0]:
# # What is COALESCE() and REPARTITION()?
# coalesce() is used to reduce the number of partitions in a DataFrame. It takes a single argument, which is the number of partitions to reduce to. It is a narrow transformation because it minimizes data movement and does not shuffle data.

# repartition() is used to increase or decrease the number of partitions in a DataFrame. It takes a single argument, which is the number of partitions to repartition to. It is a wide transformation because it involves shuffling data across the cluster.

### 21. When to use CACHE() and PERSIST()? What is the difference?

In [0]:
# 21. When to use CACHE() and PERSIST(). What is the difference?
# Use cache() when you want to store a DataFrame or RDD in `memory only` (default storage level).
# Use persist() when you want to specify a different storage level (e.g., MEMORY_AND_DISK, DISK_ONLY).
# Both are used to speed up iterative and interactive workloads by avoiding recomputation.
# Example:
# df.cache()  # stores in memory only
# df.persist(StorageLevel.MEMORY_AND_DISK)  # stores in memory and spills to disk if needed
# when you say disk, it refers to the local disks of the worker nodes (executors), not the driver.


### What is the importance of PARTITIONS in Pyspark?

In [0]:
# Partitions in PySpark determine how data is distributed across the cluster.
# They enable parallel processing, improve performance, and optimize resource utilization.
# Proper partitioning can reduce data shuffling and speed up operations.
# However, for small tables (<1TB), Databricks recommends not partitioning.
# Use coalesce() to reduce and repartition() to increase/change the number of partitions.

# Example: Checking number of partitions
print(df.rdd.getNumPartitions())

### 22. You have a dataset containing the names of employees and their departments. You need to find the department with the most employees.

In [0]:
data = [("Alice", "HR"), ("Bob", "Finance"), ("Charlie", "HR"), ("David", "Engineering"), ("Eve", "Finance")]
columns = ["employee_name", "department"]

df = spark.createDataFrame(data, columns)
# df.display()

**Solution**

In [0]:
df = df.groupBy("department").agg(count("employee_name").alias('total_employees'))
w = Window.orderBy(col('total_employees').desc())
df.withColumn('rank', dense_rank().over(w)).filter(col("rank") == 1).select("department", "total_employees").display()


# dept_counts = df.groupBy("department").agg(count("employee_name").alias("total_employees"))
# w = Window.orderBy(col("total_employees").desc())
# dept_counts.withColumn("rank", dense_rank().over(w)).filter(col("rank") == 1).select("department", "total_employees").display()

### 23. While processing sales data, you need to classify each transaction as either 'High' or 'Low' based on its amount. How would you achieve this using a when condition

In [0]:
data = [("product1", 100), ("product2", 300), ("product3", 50)]
columns = ["product_id", "sales"]

df = spark.createDataFrame(data, columns)
df.display()

**Solution**

In [0]:
df.withColumn('price_category', when(col('sales') > 50, 'High').otherwise('low')).display()

### 24. While analyzing a large dataset, you need to create a new column that holds a timestamp of when the record was processed. How would you implement this and what can be the best USE CASE?

In [0]:
data = [("product1", 100), ("product2", 200), ("product3", 300)]
columns = ["product_id", "sales"]

df = spark.createDataFrame(data, columns)
df.display()

**Solution**

In [0]:
df.withColumn('processing_time', current_timestamp()).display()

### 25. You need to register this PySpark DataFrame as a temporary SQL object and run a query on it. How would you achieve this?

In [0]:
data = [("product1", 100), ("product2", 200), ("product3", 300)]
columns = ["product_id", "sales"]

df = spark.createDataFrame(data, columns)
df.display()

**Solution**

In [0]:
df.createOrReplaceTempView("sales")

### 26. You need to register this PySpark DataFrame as a temporary SQL object and run a query on it (FROM DIFFERENT NOTEBOOKS AS WELL)?

**Solution**

In [0]:
df.createOrReplaceGlobalTempView("sales_gloabal_view")

### 27. You need to query data from a PySpark DataFrame using SQL, but the data includes a nested structure. How would you flatten the data for easier querying?

In [0]:
data = [("product1", {"price": 100, "quantity": 2}), 
        ("product2", {"price": 200, "quantity": 3})]
columns = ["product_id", "product_info"]

df = spark.createDataFrame(data, columns)
df.display()

**Solution**

**Solution**

In [0]:
df.select("product_id", "product_info.price", "product_info.quantity").createOrReplaceTempView("product_info")

### 28. You are ingesting data from an external API in JSON format where the schema is inconsistent. How would you handle this situation to ensure a robust pipeline?

In [0]:
# df = spark.read.format('json').option('mergeSchema',True).load("/databricks-datasets/structured-streaming/events")

### 29. While reading data from Parquet, you need to optimize performance by partitioning the data based on a column. How would you implement this?

In [0]:
# df.write.format("parquet").mode("overwrite").partitionBy('category').save()

### 30. You are working with a large dataset in Parquet format and need to ensure that the data is written in an optimized manner with proper compression. How would you accomplish this?

In [0]:
# Snappy is the optimal and default compression for Parquet files in Spark/Databricks due to its balance of speed and compression ratio.
# Other supported types: gzip, lzo, brotli, lz4, zstd, uncompressed.

### 31. Your company uses a large-scale data pipeline that reads from Delta tables and processes data using complex aggregations. However, performance is becoming an issue due to the growing dataset size. How would you optimize the performance of the pipeline?

In [0]:
# optimize tablename zorder by (col1)
# optimize will coalesce the partition into larger files instead of lot of small part files. if z order specidied those also taken into consideration when coalscing and update the statistics in delta logs

### What are broadcast variables, and why are they used?

In [0]:
# Broadcast variables in Spark allow you to efficiently share large, read-only data (like lookup tables) across all worker nodes, minimizing data transfer and memory usage. They are used to avoid sending the same data with every task, improving performance when the same data is needed by multiple tasks.

### Difference between df.show() and df.collect()

In [0]:
# - `df.show()`: Displays the top rows of the DataFrame in a tabular format in the notebook or console. It is used for quick visualization and does not return the data.
# - `df.collect()`: Retrieves all rows of the DataFrame as a list of Row objects to the driver program. It is used to access the actual data and can cause memory issues if the DataFrame is large.

### What is lazy evaluation?

Lazy evaluation in Spark means that transformations on DataFrames are not executed immediately. Instead, Spark builds a logical plan of operations and waits until an action (like `show()`, `collect()`, or writing data) is called to trigger computation. This allows Spark to optimize the execution plan for better performance.

### What is AQE in Spark and why is it useful?

Adaptive Query Execution (AQE) in Spark is a feature that dynamically optimizes query plans during execution based on runtime statistics. AQE enables Spark to choose better physical strategies, optimize partition sizes, and handle data skew, leading to improved performance and resource utilization, especially for complex or unpredictable workloads.

- Dynamic Partition Pruning
- Join Strategies Optimizations
- Dynamically Optimizies Skewness

### How do you handle the Skewed data in Pyspark?


Handling skewed data in PySpark:

- **Salting:** Add a random "salt" value to the skewed key to distribute records across partitions, then aggregate results after processing.
- **Broadcast Join:** Use `broadcast()` for small tables in joins to avoid shuffling large skewed tables.
- **Adaptive Query Execution (AQE):** Enable AQE (`spark.sql.adaptive.enabled = true`) to let Spark optimize join strategies and handle data skew automatically.
- **Repartitioning:** Use `repartition()` or `coalesce()` to redistribute data more evenly across partitions.
- **Filtering Hot Keys:** Process skewed keys separately or filter them out for special handling.

Example: Salting technique for a skewed join

python
from pyspark.sql.functions import col, rand

**Add salt to the skewed key**
df_skewed = df.withColumn("salt", (rand() * 10).cast("int"))
df_small = df_small.withColumn("salt", col("salt"))

**Join on both key and salt**
result = df_skewed.join(df_small, ["key", "salt"])

### 43. You are processing sales data. Group by product categories and create a list of all product names in each category.

In [0]:
data = [("Electronics", "Laptop"), ("Electronics", "Smartphone"), ("Furniture", "Chair"), ("Furniture", "Table")]
columns = ["category", "product"]
df = spark.createDataFrame(data, columns)
df.display()

In [0]:
from pyspark.sql.functions import *
df.groupBy("category").agg(collect_list("product")).display()

### 44. You are analyzing orders. Group by customer IDs and list all unique product IDs each customer purchased.

In [0]:
data = [(101, "P001"), (101, "P002"), (102, "P001"), (101, "P001")]
columns = ["customer_id", "product_id"]
df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df.groupBy("customer_id").agg(collect_set("product_id").alias("product_ids")).display()

### 45. For customer records, combine first and last names only if the email address exists.

In [0]:
data = [("John", "Doe", "john.doe@example.com"), ("Jane", "Smith", None)]
columns = ["first_name", "last_name", "email"]
df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df.withColumn('fullname', when(col("email").isNotNull(), 
                               concat(col("first_name"), lit("-"), col("last_name"))
                               )
              ).display()


### 46. You have a DataFrame containing customer IDs and a list of their purchased product IDs. Calculate the number of products each customer has purchased.

In [0]:
data = [
    (1, ["prod1", "prod2", "prod3"]),
    (2, ["prod4"]),
    (3, ["prod5", "prod6"]),
]
myschema = "customer_id INT ,product_ids array<STRING>"

df = spark.createDataFrame(data, myschema)
df.display()

In [0]:
df.withColumn("product_count", size(col("product_ids"))).display()

### 47. You have employee IDs of varying lengths. Ensure all IDs are 6 characters long by padding with leading zeroes.

In [0]:
data = [
    ("1",),
    ("123",),
    ("4567",),
]
schema = ["employee_id"]

df = spark.createDataFrame(data, schema)
df.display()

In [0]:
df.withColumn('employee_id', lpad(col("employee_id"), 6, "0")).display()

### 48. You need to validate phone numbers by checking if they start with "91"

In [0]:
data = [
    ("911234567890",),
    ("811234567890",),
    ("912345678901",),
]
schema = ["phone_number"]

df = spark.createDataFrame(data, schema)
df.display()

In [0]:
df.withColumn('flag', when(col("phone_number").startswith('91'),True).otherwise(False)
              ).display()

### 49. You have a dataset with courses taken by students. Calculate the average number of courses per student.

In [0]:
data = [
    (1, ["Math", "Science"]),
    (2, ["History"]),
    (3, ["Art", "PE", "Biology"]),
]
schema = ["student_id", "courses"]

df = spark.createDataFrame(data, schema)
df.display()

In [0]:
df.withColumn('no_course', size('courses')).groupBy().agg(avg('no_course').alias('average_no_of_courses')).display()

### 50. You have a dataset with primary and secondary contact numbers. Use the primary number if available; otherwise, use the secondary number.

In [0]:
data = [
    (None, "1234567890"),
    ("9876543210", None),
    ("7894561230", "4567891230"),
]
schema = ["primary_contact", "secondary_contact"]

df = spark.createDataFrame(data, schema)
df.display()

In [0]:
df.withColumn('contact', coalesce(col("primary_contact"), col("secondary_contact"))).display()

### 51. You are categorizing product codes based on their lengths. If the length is 5, label it as "Standard"; otherwise, label it as "Custom".

In [0]:
data = [
    ("prod1",),
    ("prd234",),
    ("pr9876",),
]
schema = ["product_code"]

df = spark.createDataFrame(data, schema)
df.display()

In [0]:
df.withColumn('flag', when(length(col("product_code"))==5, 'Standard').otherwise('Custom')).display()